In [ ]:
# from bs4 import BeautifulSoup
# import pandas as pd 
# from selenium import webdriver
# from selenium.webdriver.common.by import By
# from selenium.webdriver.support.wait import WebDriverWait
# from selenium.webdriver.support import expected_conditions as EC
# from selenium.webdriver.common.keys import Keys
# from selenium.webdriver import ActionChains
# from selenium.webdriver.common.actions.action_builder import ActionBuilder
# from pyotp import * # MAY NOT NEED THIS WHEN IT COMES TO THE HEADLESS ACCOUNT
# from login import login
# import time
# from html_config import status, safer_template, preamble



ModuleNotFoundError: No module named 'bs4'

In [ ]:
# # Set up selenium driver
# driver = webdriver.Chrome()
# wait = WebDriverWait(driver,10)

# # Login to ESRI
# login(driver, wait)


In [ ]:
# # Go to a theme and try to edit html 
# theme_url = 'https://philly-stat-360-hub-1-1-phl.hub.arcgis.com/pages/safer/edit'
# driver.get(theme_url)
# wait = WebDriverWait(driver,10)

# # Try to click the area to edit 
# edit_area = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ember102')))
# edit_area.click()

# # click the edit in html button
# edit_area = wait.until(EC.element_to_be_clickable((By.CSS_SELECTOR, '#ember206 > div.text-card-bottom-editor-bar > button')))
# edit_area.click()

# # Try to click into the edit box 
# editing = wait.until(EC.presence_of_element_located((By.CSS_SELECTOR, '#ember210 > pre > textarea')))
# time.sleep(1)
# editing.send_keys(Keys.CONTROL + "a")

# # Now need to paste in the information from the table 
# editing.send_keys('Hello')


# # ActionChains(driver) \
# #     .double_click(editing) \
# #     .perform()

# Really gonna start here since switching to API route

In [2]:
import json
import requests
import urllib.parse
from urllib.parse import urlencode
from utils import generate_token, fetch_dashboard_data, update_dashboard
from html_config import status, safer_template, preamble
import citygeo_secrets as cgs 

INFO: Mounted drive located at /tmpfs-secure
INFO: User "ubuntu" has permission to access mounted drive


In [ ]:
creds = cgs.get_secrets(AGO_LOGIN)
token = generate_token(creds[AGO_LOGIN][])

In [2]:
test = [10, status['meets'], 20, status['monitoring'], 30, status['near'], 40, status['improve'],
        50, status['meets'], 60, status['monitoring'], 70, status['near'], 80, status['improve'],
        90, status['meets'], 100, status['monitoring']]

test_html = preamble + safer_template.format(*test)

In [ ]:
def create_html(template, df):
    # Create list of values from dataframe 
    

<style>

.section-body .row {
  margin-left: 0;
  margin-right: 0;
}

.margin-row a {
  display: flex;
  outline: none !important;
}

.col-lg-6 {
  position: relative;
  min-height: 1px;
  padding-right: 7px;
  padding-left: 7px;
}

  .card-h {
    position: relative;
    display: -ms-flexbox;
    display: flex;
    -ms-flex-direction: column;
    flex-direction: column;
    min-width: 0;
    word-wrap: break-word;
    margin: 7px 0;
    height: 170px;
    width: 100%;
  }
  /*.h-meeting{
    box-shadow:.25rem 0rem #58c04d!important;
  }
  .h-needs{
    box-shadow:.25rem 0rem #cc3000!important;
  }
  .h-near{
    box-shadow:.25rem 0rem #f99300!important;
  }
  .h-measuring{
    box-shadow:.25rem 0rem #2176d2!important;
  }*/
  .card-body{
    padding: 0 0px 0 20px;
    flex-grow: 1;
    display: flex;
    align-items: stretch;
    background-color: #f0f0f0;
    background-clip: border-box;
    border: 1px solid #a1a1a1;
    border-radius: .35rem;
  }
  
  .card-body:hover{
    backgrou

In [1]:
import sqlalchemy as sa
from sqlalchemy import text
from config import DB_SECRET_HOST, DB_SECRET_LOGIN
import citygeo_secrets as cgs
from test_plz import USER, PASS, HOST, DATABASE, PORT
from sqlalchemy import text
import petl as etl 
import re


def connect_to_target(creds: dict, test: bool) -> sa.Engine: 
    db_creds = creds[DB_SECRET_HOST]
    creds_schema = creds[DB_SECRET_LOGIN]
    url_object = sa.URL.create(
        drivername='postgresql+psycopg',
        username=creds_schema['login'],
        password=creds_schema['password'],
        host=db_creds['host'],
        port=db_creds['port'],
        database=db_creds['database']
    )
    engine = sa.create_engine(url_object)
    engine.connect()
    return engine

def databridgeify(colname: str) -> str:
    """Reformat column names to match Databridge v2 conventions."""
    colname = colname.lower()
    colname = re.sub(r' |\/', '_', colname)
    colname = re.sub(':|#', '', colname)
    # make sure repeated underscores aren't introduced by repeated calls on same column
    colname = re.sub(r'_+', '_', colname)
    colname = re.sub(r'^_', '', colname)
    return colname

INFO: Mounted drive located at /tmpfs-secure
INFO: User "ubuntu" has permission to access mounted drive


In [2]:
url_object = sa.URL.create(
        drivername='postgresql+psycopg',
        username=USER,
        password=PASS,
        host=HOST,
        port=PORT,
        database=DATABASE
    )
engine = sa.create_engine(url_object)

q = '''SELECT metricid,
       parentmetricid,
       metric,
       period,
       periodstartdate,
       value,
       target,
       unit,
       unitlabel,
       targetmet,
       targettype,
       benchmark,
       comments,
       objectid
FROM philly_stat_360.metrics'''

with engine.connect() as conn: 
    results = conn.execute(text(q))
    rows = results.fetchall()
    header = [databridgeify(key) for key in results.keys()]
    rows_dict = list(dict(zip(header, list(row))) for row in rows)
    rows_obj = etl.fromdicts(rows_dict, header=header)
    df = rows_obj.todataframe()



,metricid,parentmetricid,metric,period,periodstartdate,value,target,unit,unitlabel,targetmet,targettype,benchmark,comments,objectid
0,373,NaN,Animals placed into foster care,January 2022,2022-01-01,25,None,count,Fosters,NaN,None,None,None,6971
1,373,NaN,Animals placed into foster care,February 2022,2022-02-01,26,None,count,Fosters,NaN,None,None,None,6972
2,92,89.0,Illegal Dumping - Open and On Time,August 2025,2025-08-01,338,None,Count,Open and On Time,NaN,None,None,None,15996
3,93,89.0,Illegal DUmping - Open and Overdue,August 2025,2025-08-01,28,None,Count,Open and Overdue,NaN,None,None,None,15997
4,373,NaN,Animals placed into foster care,March 2022,2022-03-01,52,None,count,Fosters,NaN,None,None,None,6973
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2938,366,NaN,total number supported with internet access by...,FY2024 Q4,2024-04-01,593,None,Count,Residents Supported,0.0,None,None,None,15684
2939,366,NaN,total number supported with internet access by...,FY2025 Q1,2024-07-01,637,None,Count,Residents Supported,0.0,None,None,None,15685
2940,366,NaN,total number supported with internet access by...,FY2025 Q2,2024-10-01,599,None,Count,Residents Supported,0.0,None,None,None,15686
2941,366,NaN,total number supported with internet access by...,FY2025 Q3,2025-01-01,536,None,Count,Residents Supported,0.0,None,None,None,15687
